In [1]:
import json
import time
import os
import re
from tqdm.notebook import tqdm
import wikipedia
from wikipedia.exceptions import WikipediaException, DisambiguationError, PageError

In [2]:
# Load the annotated queries data from JSON file
def load_queries(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
    return data['queries']

# Function to extract page title from Wikipedia URL
def extract_title_from_url(url):
    # Extract the page title from the URL path
    parts = url.split('/')
    title = parts[-1]
    # Replace underscores with spaces and URL decode
    title = title.replace('_', ' ')
    return title

# Function to extract content from Wikipedia using the wikipedia library
def extract_wikipedia_content(url):
    try:
        # Extract the title from the URL
        title = extract_title_from_url(url)
        
        # Add a small delay to be respectful
        time.sleep(0.5)
        
        # Get the page content
        page = wikipedia.page(title, auto_suggest=False)
        content = page.content
        
        # Clean the content
        content = re.sub(r'\n+', ' ', content)  # Replace newlines with spaces
        content = re.sub(r'\s+', ' ', content)  # Normalize whitespace
        content = content.strip()
        
        return content, page.url
    except DisambiguationError as e:
        print(f"Disambiguation error for {title}: {e}")
        return "", url
    except PageError as e:
        print(f"Page not found for {title}: {e}")
        return "", url
    except WikipediaException as e:
        print(f"Wikipedia API error for {title}: {e}")
        return "", url
    except Exception as e:
        print(f"Error extracting content for {title}: {str(e)}")
        return "", url


In [ ]:

# Function to build dataset with highly relevant documents
def build_dataset(queries, output_path, min_relevance=3):
    dataset = []
    
    # Process each query
    for query in tqdm(queries, desc="Processing queries"):
        if 'documents' not in query:
            print(f"Skipping query ID {query.get('id', 'unknown')}: No documents found")
            continue
            
        query_id = query.get('id')
        query_text = query.get('query', '')
        narrative = query.get('narrative', '')
        
        # Extract documents with relevance >= min_relevance
        relevant_docs = []
        for doc in query['documents']:
            relevance = doc.get('relevance_score')
            if relevance is not None and relevance >= min_relevance:
                doc_url = doc.get('url', '')
                doc_title = doc.get('title', '')
                
                if not doc_url:
                    print(f"Skipping document {doc_title}: No URL provided")
                    continue
                
                # Extract content from Wikipedia
                content, actual_url = extract_wikipedia_content(doc_url)
                
                if content:
                    relevant_docs.append({
                        'title': doc_title,
                        'url': actual_url,
                        'content': content,
                        'relevance_score': relevance
                    })
        
        # Only add the query if it has relevant documents
        if relevant_docs:
            dataset.append({
                'query_id': query_id,
                'query': query_text,
                'narrative': narrative,
                'documents': relevant_docs
            })
    
    # Save the dataset as JSON
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump({'dataset': dataset}, f, indent=2, ensure_ascii=False)
    
    print(f"Dataset saved to {output_path}")
    return dataset

# Main execution
json_path = 'ManualAnnotatedQueries.json'
output_path = 'WikipediaRelevantDocs.json'

# Check if the file already exists
if os.path.exists(output_path):
    print(f"Warning: {output_path} already exists. Running this will overwrite it.")
    proceed = input("Do you want to proceed? (y/n): ")
    if proceed.lower() != 'y':
        print("Operation cancelled.")
        import sys
        sys.exit(0)

# Load queries and build dataset
queries = load_queries(json_path)
print(f"Loaded {len(queries)} queries from {json_path}")

# Build the dataset with documents that have relevance score >= 3
dataset = build_dataset(queries, output_path, min_relevance=3)

# Print some statistics
total_docs = sum(len(item['documents']) for item in dataset)
print(f"Total queries with relevant documents: {len(dataset)}")
print(f"Total documents extracted: {total_docs}")
print(f"Average documents per query: {total_docs / len(dataset) if len(dataset) > 0 else 0:.2f}")

# Display first query and document as sample (if available)
if dataset and dataset[0].get('documents'):
    sample_query = dataset[0]
    sample_doc = sample_query['documents'][0]
    
    print("\nSample Query:")
    print(f"ID: {sample_query.get('query_id')}")
    print(f"Query: {sample_query.get('query')}")
    print(f"Narrative: {sample_query.get('narrative', '')[:100]}...")
    
    print("\nSample Document:")
    print(f"Title: {sample_doc.get('title')}")
    print(f"URL: {sample_doc.get('url')}")
    print(f"Content (first 200 chars): {sample_doc.get('content', '')[:200]}...")